In [1]:
from collections import namedtuple
import csv
import math
import numpy as np
import matplotlib.pyplot as plt

# 室内側へ流入する吸収日射量を計算するモジュール

## 1. Functions  

$ N_{lay} $層の面材からなるグレージング複合体の熱抵抗と層$j$の吸収日射量から室内側へ流入する吸収日射量を計算する。

### 入力値  

$ I_{\alpha,j} $ ：層$j$での吸収日射量（$0$～$N_{lay}-1$） (W/m<sup>2</sup>)  
$ R_{glz,k} $ ：グレージング複合体の$k$番目の熱抵抗（$0$番目を屋外側表面熱伝達抵抗、$2･j$番目をグレージング層$j-1$番目と$j$番目の間の中間層の熱抵抗、グレージング層の熱抵抗、$2･j+1$番目をグレージング層$j$番目の熱抵抗、$2･N_{lay}$番目を室内側表面熱伝達抵抗とした直列の熱抵抗）(m<sup>2</sup>･K/W)   

### 出力値  

$ I_{qi} $ ：室内側へ流入する吸収日射量 (W/m<sup>2</sup>)  

### 計算方法  

#### データ構造と値の範囲  
$ I_{\alpha} $・$ R_{glz} $の組み合わせを入力条件として1つの変数（タプル）で表し"qin_input"とする。  


In [2]:
class qin_input(namedtuple('qin_input','qin_ia qin_rglz')):
    __slots__ = ()

#qin_ia：グレージング各層の吸収日射量(W/m2)
#qin_rglz：直列に並べたグレージング複合体の熱抵抗[(m・K2)/W]

#### 各層の吸収日射熱が室内側へ再放出される割合

（参考：JIS A 2103：2014 5.4.1 計算基礎式）

層$j$の面材の吸収日射熱が室内側へ再放出される割合$N_{in,j}$の計算方法を以下に示す。  

$ \begin{align} N_{in,j} = \frac {\sum_{k=0}^{2･j} R_{glz,k} + \frac {R_{glz,2･j+1}}{2}}{\sum_{k=0}^{2･N_{lay}} R_{glz,k}} \end{align}$ ･･･････････････････････････････････････････････････････････････････････ (5.1.4.5-a)  

#### 室内へ流入する吸収日射量

（参考：JIS A 2103：2014 5.4.2 計算基礎式）

室内へ流入する吸収日射熱量$I_{qi}$の計算方法を以下に示す。  

$ \begin{align} I_{qi} = \sum_{j=0}^{N_{lay}-1} I_{\alpha,j}･N_{in,j} \end{align}$ ････････････････････････････････････････････････････････････････････････････････ (5.1.4.5-b)  


In [3]:
def Inward_Heat_Transfer(L):

    sum_R = 0.
    R_in = np.zeros(len(L.qin_ia))
    for j in range(len(L.qin_ia)):
        sum_R += L.qin_rglz[2 * j] + L.qin_rglz[2 * j + 1]
        R_in[j] = sum_R -  L.qin_rglz[2 * j + 1] / 2.

    R_tot = sum_R + L.qin_rglz[2 * len(L.qin_ia)]
    N_in = np.zeros(len(L.qin_ia))
    r_qin = 0.
    for j in range(len(L.qin_ia)):
        N_in[j] = R_in[j] / R_tot
        r_qin += L.qin_ia[j] * N_in[j]
        
    return r_qin

#### Example

In [4]:
if __name__ == '__main__':

    qin_ia =[109.03733774, 84.53584395, 63.94106107]
    qin_rglz = [0.04178292, 0.003, 0.1761019, 0.003, 0.16618873, 0.003, 0.12007248]

    qin_in = qin_input(qin_ia, qin_rglz)
    I_qi = Inward_Heat_Transfer(qin_in)

    print (I_qi)

94.62524942603363
